# soynlp로 자연어 처리
* https://github.com/lovit/soynlp

In [1]:
# 출력이 너무 길어지지 않게하기 위해 찍지 않도록 했으나 
# 실제 학습 할 때는 아래 두 줄을 주석처리 하는 것을 권장한다.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

0.21.0
1.14.0


In [3]:
petitions = pd.read_csv('data/petition_sampled.csv')
# 데이터의 크기가 어느정도인지 본다.
petitions.shape

(8029, 8)

In [4]:
petitions.head()

,article_id,start,end,answered,votes,category,title,content
0,58,2017-08-19,2017-11-17,0,21,일자리,국토교통부와 한국주택협회가 행한 부당한 행위와 권력남용에 대한 내용을 청원드립니다.,안녕하세요? 존경하고 지지하는 문재인 대통령님!\n저는 성남시 분당구 정자동 주택전...
1,63,2017-08-20,2017-09-04,0,1,보건복지,살려주세요..,안녕하십니까?\n저는 올해 63세된 홀로 사는 늙은 여자입니다...\n작년 중복날 ...
2,136,2017-08-20,2017-11-18,0,4,육아/교육,고등학교 교육 내용 수준을 낮춰주시고 실용적인 내용을 담아주세요!,저는 광주에 사는 중3 학생입니다. 고등학교 가기 직전의 학년이라 어느 때보다 고등...
3,141,2017-08-20,2017-08-27,0,0,기타,한국문화에 창조적요소를 심자,안녕하십니까\n저는 92년 한국을 알게된 종국동포 입니다.\n[저는 한 중소기업에...
4,148,2017-08-20,2017-11-18,0,7,외교/통일/국방,다문화정책 및 할랄 인증 제도,대한민국과 국민을 위해 밤낮 없이 수고하시는 대통령을 비롯한 위정자 분들께\n대한민...


In [5]:
petitions_content = ' '.join(str(petitions['content']))

In [6]:
class Sentences:
    def __init__(self, fname):
        self.fname = fname
        self.length = 0
    def __iter__(self):
        for doc in self.fname:
            doc = doc.strip()
            if not doc:
                continue
            for sent in doc.split(' '):
                yield sent
    def __len__(self):
        if self.length == 0:
            for doc in self.fname:
                doc = doc.strip()
                if not doc:
                    continue
                self.length += len(doc.split(' '))
        return self.length

In [7]:
corpus_fname = petitions['title']
sentences = Sentences(corpus_fname)
print('num sentences = %d' % len(sentences))

num sentences = 37956


In [8]:
%%time
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_count=100,
                               min_cohesion_forward=0.05, 
                               min_right_branching_entropy=0.0)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.114 Gb
all cohesion probabilities was computed. # words = 101
all branching entropies was computed # words = 91
all accessor variety was computed # words = 91
CPU times: user 548 ms, sys: 55.7 ms, total: 604 ms
Wall time: 722 ms


In [9]:
len(words)

198

words는 {word:Score} 형식의 dictionary입니다. Score는 soynlp/word.py에 구현되어있는 namedtuple입니다.

In [10]:
print('type: %s\n' % type(words['가상화폐']))
print(words['가상화폐'])

type: <class 'soynlp.word._word.Scores'>

Scores(cohesion_forward=0.7862404931722864, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=261, rightside_frequency=0)


In [11]:
print('type: %s\n' % type(words['청원']))
print(words['청원'])

type: <class 'soynlp.word._word.Scores'>

Scores(cohesion_forward=0.5051440329218106, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=-0.0, left_accessor_variety=0, right_accessor_variety=1, leftside_frequency=491, rightside_frequency=0)


In [12]:
def word_score(score):
    import math
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (word, 
                                   score.leftside_frequency, 
                                   score.cohesion_forward,
                                   score.right_branching_entropy
                                  ))

단어   (빈도수, cohesion, branching entropy)

청소년     (346, 0.597, 0.692)
폐지     (570, 0.857, -0.000)
합니다     (195, 0.823, -0.000)
출국금지     (321, 0.809, 0.000)
처벌     (251, 0.789, 0.000)
바랍니다     (102, 0.787, 0.000)
가상화폐     (261, 0.786, 0.000)
반대합니다     (166, 0.780, 0.000)
반대     (344, 0.766, -0.000)
만들어     (102, 0.749, 0.000)
부탁드립니다     (101, 0.747, 0.000)
이명박     (537, 0.731, 0.000)
합니     (210, 0.729, -0.000)
가상화     (282, 0.725, -0.000)
합니다.     (106, 0.717, 0.000)
해주세요     (215, 0.715, 0.000)
소년법     (179, 0.712, 0.000)
청원합니다     (238, 0.703, -0.000)
문재인     (139, 0.685, 0.000)
금지     (160, 0.675, 0.000)
청원합니다.     (130, 0.669, 0.000)
주세요     (257, 0.665, 0.000)
출국     (400, 0.660, -0.000)
만들     (119, 0.654, -0.000)
청소년보호법     (112, 0.649, 0.000)
조두순     (186, 0.624, 0.000)
요청     (137, 0.614, 0.000)
국회의원     (100, 0.533, 0.000)
청원     (491, 0.505, -0.000)
부탁드     (108, 0.498, -0.000)


In [13]:
score

Scores(cohesion_forward=0.4982728791224398, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=-0.0, left_accessor_variety=0, right_accessor_variety=1, leftside_frequency=108, rightside_frequency=0)

In [14]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

In [15]:
sample_title = petitions['title'][2]
sample_title

'고등학교 교육 내용 수준을 낮춰주시고 실용적인 내용을 담아주세요!'

### RegexTokenizer

In [16]:
print(tokenizer.tokenize(sample_title))

['고등학교', '교육', '내용', '수준을', '낮춰주시고', '실용적인', '내용을', '담아주세요', '!']


### Normalizer

In [17]:
from soynlp.normalizer import *

In [18]:
# 동작이 예제와 다름
repeat_normalize('와하하하하하핫', n_repeats=2)

'와하하하하하핫'

In [19]:
emoticon_normalize('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ쿠ㅜㅜㅜㅜㅜㅜ', n_repeats=3)

'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅜㅜㅜㅜㅜㅜㅜ'

In [20]:
from soynlp.tokenizer import LTokenizer

In [21]:
tokenizer = LTokenizer(scores=score)
tokenizer